In [1]:
#bring imports here
import json
import requests
import numpy as np
import pandas as pd
import os
import sys
from time import sleep

 ILThermo's 4-letter tag for conductivity is GtfG. Performed a search for all conductivity data with keyword as 'Deep Eutectic Solvents' and returned a decent amount of data.

URL for the data is https://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=Deep%20eutectic%20solvent&prp=GtfG

In [13]:
paper_url = "https://ilthermo.boulder.nist.gov/ILT2/ilsearch?cmp=&ncmp=0&year=&auth=&keyw=Deep%20eutectic%20solvent&prp=GtfG"
 
r = requests.get(paper_url)
header = r.json()['header']
papers = r.json()['res']
i = 1 
data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
for paper in papers[:]: 
    
    r = requests.get(data_url.format(paper_id=paper[0]))
    data = r.json()['data']
    with open("nist_data/%s.json" % i, "w") as outfile: 
        json.dump(r.json(), outfile)
    #then do whatever you want to data like writing to a file
    sleep(0.5) #import step to avoid getting banned by server
    i += 1

The data was successfully scraped from ILThermo, below I have shown the first file. A total of 31 files were scraped.

In [3]:
with open('nist_data/conductivity/1.json') as f:
    data = json.load(f)
    
print(data)

{'data': [[['298.15'], ['0.2'], ['101'], ['0.0116', '0.0006']], [['298.15'], ['0.25'], ['101'], ['0.0103', '0.0006']], [['298.15'], ['0.34'], ['101'], ['0.0062', '0.0006']], [['303.15'], ['0.2'], ['101'], ['0.0198', '0.0006']], [['303.15'], ['0.25'], ['101'], ['0.0172', '0.0006']], [['303.15'], ['0.34'], ['101'], ['0.0124', '0.0006']], [['308.15'], ['0.2'], ['101'], ['0.037', '0.001']], [['308.15'], ['0.25'], ['101'], ['0.0319', '0.0006']], [['308.15'], ['0.34'], ['101'], ['0.0186', '0.0006']], [['313.15'], ['0.2'], ['101'], ['0.041', '0.001']], [['313.15'], ['0.25'], ['101'], ['0.0394', '0.0006']], [['313.15'], ['0.34'], ['101'], ['0.0277', '0.0006']], [['318.15'], ['0.2'], ['101'], ['0.0607', '0.0006']], [['318.15'], ['0.25'], ['101'], ['0.0549', '0.0006']], [['318.15'], ['0.34'], ['101'], ['0.0405', '0.0006']], [['323.15'], ['0.2'], ['101'], ['0.0816', '0.0006']], [['323.15'], ['0.25'], ['101'], ['0.0719', '0.0006']], [['323.15'], ['0.34'], ['101'], ['0.0496', '0.0006']], [['328.15'

Now the code below will convert the json files to a dataframe, this can be configured to convert and append multiple json files at once by adding in the commented out for loop portion at the very beginning and adding i as a string in the path.

In [2]:
outer_old = pd.DataFrame()
outer_new = pd.DataFrame()

for i in range(31):
    with open("/Users/jaime/Desktop/nist_data/conductivity/%s.json" % str(i+1)) as json_file:

        #grab data, data headers (names), the salt name
        json_full = json.load(json_file)                    
        json_data = pd.DataFrame(json_full['data'])
        json_datanames = np.array(json_full['dhead'])        # make names into array to add as columns headers for df
        json_data.columns =  json_datanames
        json_saltname = pd.DataFrame(json_full['components'])#components section contains names of DES components
        #print(json_saltname['name'])                #grabbing the HBD and HBA 

        inner_old = pd.DataFrame()
        inner_new = pd.DataFrame()

        #loop through the columns of the data, note that some of the 
        #json files are missing pressure data. 
        for indexer in range(len(json_data.columns)):        
            grab=json_data.columns[indexer]
            list = json_data[grab]
            my_list = [l[0] for l in list]
            dfmy_list = pd.DataFrame(my_list)
            dfmy_list.columns = [json_datanames[indexer][0]]
            inner_new = pd.concat([dfmy_list, inner_old], axis=1)
            inner_old = inner_new

        #print(inner_old.columns)
        
        
        #add the MW for HBD and HBA
        for i in range(len(json_saltname['name'])):
            if 'chloride' in json_saltname['name'][i] or 'bromide' in json_saltname['name'][i]:
                inner_old['HBA_MW']=json_saltname['mw'][i]
            else:
                inner_old['HBD_MW']=json_saltname['mw'][i]
                

        #add the DES components, i.e. HBA and HBD
        # they are not always listed in the same order on nist data, i.e., HBA always first. Will figure out later.
        for i in range(len(json_saltname['name'])):
            if 'chloride' in json_saltname['name'][i] or 'bromide' in json_saltname['name'][i]:
                inner_old['HBA']=json_saltname['name'][i]
            else:
                inner_old['HBD']=json_saltname['name'][i]
                
        #loop through the column names of the dataframe
        for j in range(len(inner_old.columns)):
            #if the words Mole fraction and a halogen are contained, values are correct and no value editing 
            #necessary and column is simply renamed to HBA mole fraction.
            if 'Mole fraction' in inner_old.columns[j] and 'chloride' in inner_old.columns[j] or 'Mole fraction' in inner_old.columns[j] and 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
            #if the words Mole Ratio and a halogen are contained, dataset was mislabeled but values are correct.
            #only need to rename column to HBA mole fraction.
            elif 'Mole ratio' in inner_old.columns[j] and 'chloride' in inner_old.columns[j] or 'Mole ratio' in inner_old.columns[j] and 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
            #if the words mole ratio are present, but no halogens, the ratio of the HBD is displayed and needs
            #to be changed to HBA mole fraction. First relabel the colum as HBA mole fraction.
            elif 'Mole ratio' in inner_old.columns[j] and not 'chloride' in inner_old.columns[j] or 'Mole ratio' in inner_old.columns[j] and not 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
                #apparently the numbers are strings so change to integer. May need to do this for every other column
                inner_old['HBA Mole Fraction'] = inner_old['HBA Mole Fraction'].astype(int)
                #next make an empty list that will hold all the new HBA mole fractions
                mole_fractions_list = []
                #loop through every HBD ratio in the column
                for k in range(len(inner_old['HBA Mole Fraction'])):
                    #Calculate the HBA mole fraction from every HBD ratio and append to the list
                    mole_fractions_list.append(1/(1+inner_old['HBA Mole Fraction'][k]))
                #finally make the list the new mole fraction column in the dataframe
                inner_old['HBA Mole Fraction'] = mole_fractions_list
            #in the last case, if the word mole fraction is present but not a halogen, HBD mole fraction is displayed.
            #Follow simialr process as before
            elif 'Mole fraction' in inner_old.columns[j] and not 'chloride' in inner_old.columns[j] or 'Mole fraction' in inner_old.columns[j] and not 'bromide' in inner_old.columns[j]:
                inner_old = inner_old.rename(columns={inner_old.columns[j]:'HBA Mole Fraction'})
                #convert to float instead since it is a decimal
                inner_old['HBA Mole Fraction'] = inner_old['HBA Mole Fraction'].astype(float)
                #empty list
                mole_fractions_list = []
                #loop through column
                for k in range(len(inner_old['HBA Mole Fraction'])):
                    #subtract 1 from HBD mole fraction to get HBA mole fraction and append to list
                    mole_fractions_list.append(1 - inner_old['HBA Mole Fraction'][k])
                #replace column   
                inner_old['HBA Mole Fraction'] = mole_fractions_list


        #add to the growing dataframe
        outer_new = pd.concat([inner_old, outer_old], axis = 0, ignore_index = True)
        outer_old = outer_new
    
outer_old.head(50)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,"Electrical conductivity, S/m",HBA,HBA Mole Fraction,HBA_MW,HBD,HBD_MW,"Pressure, kPa","Temperature, K"
0,0.049,benzyltriphenylphosphonium chloride,0.25,388.87,"1,2-ethanediol",62.07,101.3,328.15
1,0.02,benzyltriphenylphosphonium chloride,0.25,388.87,"1,2-ethanediol",62.07,101.3,338.15
2,0.0016,benzyltriphenylphosphonium chloride,0.25,388.87,"1,2-ethanediol",62.07,101.3,348.15
3,0.0015,benzyltriphenylphosphonium chloride,0.25,388.87,"1,2-ethanediol",62.07,101.3,358.15
4,0.0014,benzyltriphenylphosphonium chloride,0.25,388.87,"1,2-ethanediol",62.07,101.3,368.15
5,0.016,benzyltriphenylphosphonium chloride,0.167,388.87,glycerol,92.09,101.3,328.15
6,0.0019,benzyltriphenylphosphonium chloride,0.167,388.87,glycerol,92.09,101.3,338.15
7,0.0017,benzyltriphenylphosphonium chloride,0.167,388.87,glycerol,92.09,101.3,348.15
8,0.0015,benzyltriphenylphosphonium chloride,0.167,388.87,glycerol,92.09,101.3,358.15
9,0.0014,benzyltriphenylphosphonium chloride,0.167,388.87,glycerol,92.09,101.3,368.15


In [3]:
outer_old.dtypes

Electrical conductivity, S/m    object
HBA                             object
HBA Mole Fraction               object
HBA_MW                          object
HBD                             object
HBD_MW                          object
Pressure, kPa                   object
Temperature, K                  object
dtype: object

In [4]:
pd.DataFrame.to_csv(outer_old, path_or_buf='conductivity.csv', index=False)